In [ ]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

import os
import time
import numpy as np
import pandas as pd
import neuropythy as ny
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit



In [ ]:
# Settings
max_ecc = 8
max_CM = 1000
r2_th = 0.1
rois = ["V1", "V2", "V3", "hV4", "VO1", "VO2",
        "PHC1", "PHC2", "TO2", "TO1", "LO2", "LO1", "V3B", "V3A",
        "IPS0", "IPS1", "IPS2", "IPS3", "IPS4", "IPS5", "SPL1", "FEF"]


In [ ]:
# Get subject numbers
allsub = ny.data['hcp_lines'].subject_list

In [ ]:
path = '/home/naxos2-raid27/wong0876/pRF_project/tsv/group_tsv'
grouptsv = pd.read_csv('{}/area_concat_r2_filtered.tsv'.format(path), sep='\t')

In [ ]:
# Group ventral and dorsal for V1, V2, V3
roi_map = {
    "V1v": "V1", "V1d": "V1",
    "V2v": "V2", "V2d": "V2",
    "V3v": "V3", "V3d": "V3",
}

# Replace labels in grouptsv
grouptsv['roi'] = grouptsv['roi'].replace(roi_map)

In [ ]:
import numpy as np
import pandas as pd
from scipy.optimize import least_squares, minimize_scalar
from scipy.stats import chi2

import numpy as np
from scipy.optimize import least_squares

# === New Harvey functions on log scale ===
def harvey_func(x, c, d):
    # original scale
    return 1.0 / (c*x + d)**2

def harvey_log_func(x, c, d):
    # log-scale model: log(y) = -2 * log(c*x + d)
    return -2.0 * np.log(c*x + d)

# === Horton function ===
def horton_func(x):
    return 300 / (x + 0.75)**2

# === Residuals on log scale ===
def residuals_log(params, x, y):
    c, d = params
    return np.log(y) - harvey_log_func(x, c, d)

# === Fit on log scale with bounds for stability ===
def fit_harvey_log(x, y):
    # ensure positivity
    if np.any(y <= 0):
        raise ValueError("y must be positive for log transform")

    p0 = [0.01, 1.0]
    # bounds keep c,d reasonable
    res = least_squares(residuals_log, p0, args=(x, y),
                        loss='huber', f_scale=0.05,
                        bounds=([1e-6, 0.01], [1.0, 20.0]))
    c_hat, d_hat = res.x
    # residuals in log-domain
    fun = res.fun
    sigma2_hat = np.var(fun, ddof=2)  # variance of log-residuals
    # Jacobian of log-residuals at data points (rows: n_data, cols: params)
    # For log model r = log(y) - (-2 log(c x + d)) = log(y) + 2 log(c x + d)
    # So derivative of model m = -2 log(c x + d) => dm/dc = -2 * x / (c x + d), dm/dd = -2 * 1 / (c x + d)
    J = np.column_stack([
        -2.0 * x / (c_hat * x + d_hat),   # dm/dc  (note: derivative of m wrt c)
        -2.0 / (c_hat * x + d_hat)        # dm/dd
    ])
    # Covariance of parameters (approx): cov = (J^T J)^{-1} * sigma2
    try:
        cov_params = np.linalg.inv(J.T @ J) * sigma2_hat
    except np.linalg.LinAlgError:
        cov_params = np.diag([1e-6, 1e-6])
    return (c_hat, d_hat), cov_params, sigma2_hat, res.fun

# === Asymptotic CI computed in log-space and exponentiated ===
def asymptotic_ci_log(x_common, x_data, y_data, best_params, cov_params, sigma2_hat, alpha=0.05):
    c_hat, d_hat = best_params
    # predicted log y
    logy_fit = harvey_log_func(x_common, c_hat, d_hat)

    # derivatives of logy wrt params at prediction points
    dlogy_dc = -2.0 * x_common / (c_hat * x_common + d_hat)
    dlogy_dd = -2.0 / (c_hat * x_common + d_hat)
    J_pred = np.column_stack([dlogy_dc, dlogy_dd])

    # variance of log-prediction: var_logy = J_pred @ cov_params @ J_pred^T
    # compute vectorized:
    var_logy = np.einsum('ij,jk,ik->i', J_pred, cov_params, J_pred)

    # add observation noise if you want prediction intervals (optional)
    # var_pred_log = var_logy + sigma2_hat   # uncomment for prediction interval
    var_pred_log = var_logy

    se_log = np.sqrt(np.maximum(var_pred_log, 0.0))
    z = 1.96  # ~95% CI
    log_lower = logy_fit - z * se_log
    log_upper = logy_fit + z * se_log

    # exponentiate to get CI on original scale
    y_fit = np.exp(logy_fit)
    y_lower = np.exp(log_lower)
    y_upper = np.exp(log_upper)

    return y_fit, y_lower, y_upper

# -----------------------------
# RMSE
# -----------------------------
def compute_rmse(y_true, y_pred):
    return np.sqrt(np.mean((y_true - y_pred)**2))

# -----------------------------
# Run across ROIs
# -----------------------------
x_common = np.linspace(0, 8, 200)
roi_results = {}
roi_metrics = {}

for roi in rois:
    print(roi, end=' ')
    df_roi = grouptsv[grouptsv['roi'] == roi]
    if len(df_roi) < 6:  # skip very small ROIs
        continue

    x = df_roi['prf_ecc'].to_numpy()
    y = df_roi['pRF_CM'].to_numpy()

    # fit on log scale
    best_params, cov_params, sigma2_hat, fun = fit_harvey_log(x, y)
    c_hat, d_hat = best_params

    # compute asymptotic CI in log space (smooth, multiplicative bands)
    y_fixed, y_lower, y_upper = asymptotic_ci_log(x_common, x, y, best_params, cov_params, sigma2_hat, alpha=0.05)

    # Save results
    roi_results[roi] = {
        "x_common": x_common,
        "y_fixed": y_fixed,
        "y_lower": y_lower,
        "y_upper": y_upper,
        "params": {"c": c_hat, "d": d_hat},
        "y_horton": horton_func(x_common)
    }

    # RMSE
    y_pred = harvey_func(x, c_hat, d_hat)
    roi_metrics[roi] = {"Harvey_RMSE": compute_rmse(y, y_pred)}

# -----------------------------
# Print summary
# -----------------------------
for roi in rois:
    if roi in roi_metrics:
        print(f"{roi}: Harvey RMSE={roi_metrics[roi]['Harvey_RMSE']:.3f}")


In [ ]:
np.save('roi_results_area.npy', roi_results)
np.save('roi_metrics_area.npy', roi_metrics)